In [11]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

# Import Local funcs
import sys
sys.path.append('../')
from utils.utils import *

In [2]:
X, y = get_training()
X = X.select_dtypes(exclude=['object'])

In [4]:
# Make Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
d_train = lgb.Dataset(X_train, label=y_train)
d_test = lgb.Dataset(X_test, label=y_test, reference=d_train)

# Train Model only on Response

In [4]:
# Get indices where labels are 0
idx_0 = y_train[y_train.label == 0].index
len(idx_0)

464

In [5]:
# Remove these indices from the training set
X_train = X_train.drop(idx_0)
y_train = y_train.drop(idx_0)

In [9]:
# Rearrange labels
y_train.label = y_train.label - 1

In [10]:
print(len(X_train))
print(len(y_train))

33136
33136


In [11]:
X_train_less, X_test_less, y_train_less, y_test_less = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
d_train = lgb.Dataset(X_train_less, label=y_train_less)
d_test = lgb.Dataset(X_test_less, label=y_test_less, reference=d_train)

In [ ]:
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",

    "num_leaves": 10, #27
    # "max_depth": 5,
    "learning_rate": 0.01,
    #"n_estimators": 1000,
    #"subsample_for_bin": 200000,
    #"class_weight"='balanced',
    #"min_child_samples": 20,
    #"reg_alpha": 0.1,
    #"reg_lambda": 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    "class_weight": "balanced",               
    
    "boost_from_average": True,
    "num_class" : 5,
    "verbose": 1,
}

model = lgb.train(
    params,
    d_train,
    num_boost_round=10000,
    valid_sets=d_test,
    callbacks=[lgb.early_stopping(50),lgb.log_evaluation(10)]
)

In [17]:
lgbm = lgb.LGBMClassifier(**params, n_estimators=3000)
lgbm.fit(X_train, y_train)


/Users/zaza/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/zaza/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037546 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8403
[LightGBM] [Info] Number of data points

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boost_from_average=True,
               class_weight='balanced', feature_fraction=0.8,
               learning_rate=0.01, metric='multi_logloss', n_estimators=3000,
               num_class=5, num_leaves=10, objective='multiclass', verbose=1)

In [18]:
# Refactor predictions
y_pred_proba = lgbm.predict_proba(X_test)

In [29]:
# Add zeros to the predictions
y_pred_proba_zerod = np.insert(y_pred_proba, 0, 0, axis=1)

In [19]:
# Evaluate MultiLogLoss on Test Set
multi_log_loss = log_loss(y_test, y_pred_proba)
print(f'Multi Log Loss: {multi_log_loss:.5f}')

Multi Log Loss: 0.86551
